## Ch4. 신경망 학습

#### 4.1 데이터에서 학습한다

* 데이터에서 학습한다 = 가중치 매개변수의 값을 데이터로부터 자동으로 결정한다.

* 신경망은 사람의 판단이 개입하지 않고 기계가 모든것을 데이터로부터 판단하도록 한다.

* 기계학습문제는 데이터를 **훈련데이터(training data)** 와 **시험 데이터(test data)** 로 나눠 학습하도록 한다.

#### 4.2 손실함수
* 손실 함수(loss function): 최적의 매개변수를 찾는 평가 함수. 보통 평균제곱오차(mean square error, MSE) 혹은 교차 엔트로피 오차(cross entropy error, CEE)가 사용된다.

* 평균제곱 오차 : $N$-차원 데이터에서 신경망의 출력을 $y_k$, 정답 레이블을 $t_k$ 라 하면 평균제곱오차 $E$는 다음과 같이 정의된다.

\begin{equation*}
E= \dfrac{1}{2} \sum_{k=1}^N (y_k - t_k)^2
\end{equation*}

* 교차 엔트로피 오차 : 
\begin{equation*}
E = -\sum_{k=1}^N t_k \log y_k
\end{equation*}

만약 $t_k$ 가 one-hot encoding 이라면 실질적으로 정답일 때의 추정이다.

* 만약 $M$ 개의 데이터 셋에 대해 학습한다면 각각의 교차 엔트로피 오차에 대한 평균이 손실함수가 된다.
\begin{equation*}
E = - \dfrac{1}{M}\sum_{i=1}^M \sum_{j=1}^N t_{ij}\log y_{ij}
\end{equation*}

* 데이터 셋이 아주 많다면 일부 데이터만 선택하여 학습하고 이를 전체 데이터셋의 근사치로 사용 할 수 있다. 이 선택된 일부를 **미니배치(mini-batch)** 라 하고 이러한 학습 방법을 **미니배치 학습** 이라 한다.

In [8]:
import numpy as np

def mean_sequare_error(y, t):
    return 0.5 * np.sum((y-t)**2)

def cross_entropy_error(y, t):
    """
    y, t 가 1차원 데이터일때, 2차원으로 변경하는 부분 추가.
    """
    
    if y.ndim == 1 :
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    delta = 1e-7
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+delta))/batch_size

#### 4.4 경사하강법

* 기계학습 문제 대부분은 학습단계에서 최적의 매개변수를 찾아낸다. 신경망 역시 학습시에 최적의 매개변수 (가중치와 편향) 을 찾아내야 한다. 여기서 최적이란 *손실함수가 최소값이 될 때의 매개변수값* 이다.

* 일반적으로 손실함수는 매우 복잡한 다차원 함수이다. 이 상황에서 기울기를 잘 이용하여 최소값을 찾으려는 것이 **경사하강법**이다. 

* $n$-차원 매개변수 공간에서의 경사법을 수식으로 나타내면 다음과 같다.
\begin{equation*}
%x_i = x_i - \eta \dfrac{\partial f}{\partial x_i} \qquad i=1,\,2,\ldots,\,n \\
\mathbf{x} =\mathbf{x} - \eta (\nabla f)
\end{equation*}

여기서 $f$ 는 손실함수이며 $\eta$ 는 갱신하는 양을 나타내는데 신경망 학습에서는 **학습률(learning rate)** 라 한다. 한번의 학습으로 얼마나 학습해야 할지, 즉 매개변수 값을 얼마나 갱신하느냐를 정하는 것이 학습률이다.



In [71]:
def numerical_gradiant(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        temp_val = x[idx]
        x[idx]= temp_val+h
        fxh1=f(x)
        
        x[idx] = temp_val -h
        fxh2 = f(x)
        
        grad[idx]=(fxh1-fxh2)/(2*h)
        x[idx]= temp_val
    return grad

def gradient_decent(f, init_x, lr = 0.01, step_num = 100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradiant(f, x)
        x -= lr*grad
    return x

In [70]:
def f2(x):
    return x[0]**2 + x[1]**2

In [74]:
init_x = np.array([-3., 4.])
gradient_decent(f2, init_x, lr=0.1, step_num = 100)

array([-6.11110793e-10,  8.14814391e-10])

In [75]:
# 학습률이 너무 작을 때..
gradient_decent(f2, init_x, lr=1e-10, step_num = 100)

array([-6.11110781e-10,  8.14814374e-10])

In [76]:
# 학습률이 너무 클 때..
gradient_decent(f2, init_x, lr=3, step_num = 100)

array([ 3.44544357e+12, -5.37235148e+12])

* 신경망에서의 기울기 : 신경망에서의 기울기는 가중치 매개변수에 대한 손실함수의 기울기이다. 예를 들어 형상이 $2 \times 3$, 가중치가 $\mathbf{W}$, 손실함수가 $L$ 인 신경망을 생각해 보자.
\begin{equation*}
\mathbf{W} = \begin{bmatrix} w_{11} & w_{12} & w_{13} \\ w_{21} & w_{22} & w_{23} \end{bmatrix}
\end{equation*}
이라면 경사는 $\dfrac{\partial L}{\partial \mathbf{W}}$ 라 쓰며 다음과 같이 정의된다.

\begin{equation*}
\dfrac{\partial L}{\partial \mathbf{W}} = \begin{bmatrix} \dfrac{\partial L}{\partial w_{11}} & \dfrac{\partial L}{\partial w_{12}} & \dfrac{\partial L}{\partial w_{13}} \\ \dfrac{\partial L}{\partial w_{21}} & \dfrac{\partial L}{\partial w_{22}} & \dfrac{\partial L}{\partial w_{23}}\end{bmatrix}
\end{equation*}



#### 4.5 학습 알고리즘 구현하기

* 신경망 학습의 절차.

0. 전제 : 신경망에는 적응 가능한 가중치와 편항이 있으며, 이 가중치와 편향을 훈련 데이터를 통해 조정하는 과정을 **학습** 이라 한다.

1. 1단계 : 훈련 데이터중 일부를 무작위로 선택한다. (이렇게 선택된 데이터를 **미니배치**라 한다)

2. 2단계 : 미니배치의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실함수의 값을 가장 작게 하는 방향을 제시한다.

3. 3단계 : 가중치의 매개변수를 기울기 방향으로 아주 조금 갱신한다.

4. 4단계 : 1~3 단계를 반복한다.

* 데이터를 미니배치로 무작위로 선정하므로 **확률적 경사하강법(stochastic gradient decent, SGD)** 라 한다.

In [93]:
import scratch.ch04.two_layer_net

/Users/jiyong/development/jupyternotebook


ModuleNotFoundError: No module named 'common'

In [79]:
numerical_gradiant.__code__

<code object numerical_gradiant at 0x7ffecb86e4b0, file "<ipython-input-71-a2d1644a3410>", line 1>

In [84]:
import sys

In [85]:
import os

In [92]:
os.path

<module 'posixpath' from '/Users/jiyong/opt/anaconda3/lib/python3.7/posixpath.py'>